In [ ]:
!pip3 install transformers

In [ ]:
!pip3 install urllib3==1.25.4 # нужно для доступа к изначальной сберовской модели

In [3]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

Данные

In [6]:
!unzip "aneqs.zip" # здесь нужно указать путь к архиву с csv-файлом

Archive:  /content/drive/My Drive/aneqs.zip
  inflating: aneqs.csv               


In [7]:
aneqs = pd.read_csv('aneqs.csv', encoding='cp1251') # у меня винда :c 
aneqs.text = aneqs.text.map(lambda x: x.lower())

In [8]:
aneqs.head()

,text
0,украинские хакеры научились скачивать газ чере...
1,самый большой облом на 1 января для родителей....
2,"- катя мечтает, чтобы её хотели все мужчины, п..."
3,"- ""камасутра"" - фигня! доказано сопроматом!"
4,если вы проснулись 1 января утром и никак не с...


In [9]:
data_valid = ''
data_train = ''
#data_len = len(aneqs)
for text in aneqs.text:
  text = text + '\n'
  if len(data_train) <= 600000: # можно настроить длину тренировочного сета (каждая строчка - новый анек)
    data_train+= text
  elif len(data_valid) <= 20000: # и тестового
    data_valid +=text
  else:
    break

with open('train_file.txt', 'w', encoding='utf8') as f1:
  f1.write(data_train)
with open('test_file.txt', 'w', encoding='utf8') as f2:
  f2.write(data_valid)
  

In [10]:
train_path = 'train_file.txt'
test_path = 'test_file.txt'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2") # сберовский токенизатор

In [12]:
tokenizer.pad_token = 0
tokenizer.bos_token = 2
tokenizer.eos_token = 3

In [ ]:
from transformers import DataCollatorForLanguageModeling, LineByLineTextDataset # так как у меня анеки разделены \n, можно использовать LineByLineTextDataset

def load_dataset(train_path,test_path, tokenizer):
    train_dataset = LineByLineTextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=256)

    test_dataset = LineByLineTextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=256)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

In [14]:
assert torch.cuda.is_available(), 'у вас не находится гпу'

device = torch.device('cuda:0')

Тюнинг

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

training_args = TrainingArguments(
    output_dir="./gpt3-aneqs", 
    overwrite_output_dir=True, 
    num_train_epochs=10, 
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,  
    eval_steps = 400, 
    save_steps=800, 
    warmup_steps=500,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.prediction_loss_only=True

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()